In [10]:
import sys

sys.path.append('../src')

In [12]:
from numpy import exp, pi, sqrt
from CharFuncInverter.bohman1975 import A, B, C, D, E
import pandas as pd
import numpy as np

# Провалидируем классы, опираясь на рассчёты описанные в статье Бохмана.

В качестве характеристической функции было следущее выражение:
$$\phi(t) = (1 -it \sqrt{2})^{0.5}\cdot exp(\frac{-it}{\sqrt{2}}).$$
Параметры в примере: $N = 512$, $K = 4$, $d = \frac{0.35}{16}$

In [13]:
phi = lambda t : ((1 - 1j * t * sqrt(2)) ** (-0.5)) * exp((-1j * t) / sqrt(2))

N = 512
K = 4
d = 0.35 / 16
delta = (2 * pi) / (N * d)

Также были заданы значения функции распределения в 18 точках

In [14]:
F_exact = {
    -5.60 : 0,
    -4.55 : 0,
    -3.50 : 0,
    -2.45 : 0,
    -1.40 : 0,
    -1.05 : 0,
    -0.70 : 79586,
    -0.35 : 522700,
    0 : 682690,
    0.35 : 778553,
    0.70 : 841654,
    1.40 : 915695,
    1.75 : 937693,
    2.10 : 953678,
    3.15 : 980485,
    4.20 : 991570,
    5.25 : 996298
}
xs = F_exact.keys()

In [16]:
phi(np.array(list(xs)))

array([-0.35232467-0.03378854j, -0.31605063+0.23168238j,
       -0.09639783+0.43443868j,  0.24470505+0.46627912j,
        0.60794495+0.28502237j,  0.72351528+0.18736706j,
        0.83838663+0.08818682j,  0.94654181+0.01673443j,
        1.        +0.j        ,  0.94654181-0.01673443j,
        0.83838663-0.08818682j,  0.60794495-0.28502237j,
        0.4894611 -0.36750803j,  0.3677386 -0.42881365j,
        0.00861041-0.46792474j, -0.26157177-0.31241396j,
       -0.3613715 -0.05378073j])

Создадим экземпляры классов, которые будут аппроксимировать характеристическую функцию

In [17]:
approxs = {
    'A' : A(N, delta, phi),
    'B' : B(N, delta, phi),
    'C' : C(N, delta, phi),
    'D' : D(N, delta, phi, K),
    'E' : E(N, delta, phi, K)
}
values = []

for name in approxs:
  approx_func = approxs[name]
  values.append([(approx_func.cdf(x).real * 1e6 - F_exact[x]) for x in xs])

values = np.array(values)

In [18]:
column_names = ["A", "B", "C", "D", "E"]
res = pd.DataFrame(values.transpose(), index=xs, columns=column_names)
res

,A,B,C,D,E
-5.60,-2835.655730,-2822.021892,-2835.645013,20.786333,1.535679
-4.55,-1269.515740,-1257.417101,-1269.515725,6.848571,0.665318
-3.50,-574.605167,-564.686397,-574.605167,-4.245824,0.111002
-2.45,-260.973463,-255.150637,-260.973463,-12.100360,-2.771487
-1.40,-106.938778,-115.760781,-106.938778,-8.587604,-12.340057
-1.05,-56.835252,-88.472087,-56.835252,11.127882,-13.675321
-0.70,-4123.270736,-8451.476950,-4123.270736,-4079.488504,-8395.857056
-0.35,-117.335538,-130.953419,-117.335538,-92.985080,-85.920671
0.00,-80.103089,-70.163900,-80.103089,-71.554857,-29.446675
0.35,-61.057530,-45.994062,-61.057530,-65.543397,-6.791820


In [31]:
Total = (res['B']**2).sum()/len(res['E'])
Total

np.float64(4788445.307220343)

### Вывод
Полученные, с помощью написанных функций результаты совпадают с описанными Бохманом в статье, по модулю округления

In [32]:
class CharFuncInverter:
    def __init__(self):
        pass

    def fit(self):
        pass

    def cdf(self, x):
        pass


In [137]:
approxs = {
    'E' : BohmanE(N, delta, K),
}

values = []

for name in approxs:
  approx_func = approxs[name]
  approx_func.fit(phi)  

  print(1e6 * approx_func.cdf(list(xs)).real - np.array(list(F_exact.values())))

  values.append([(approx_func.cdf(x).real * 1e6 - F_exact[x]) for x in xs])

values = np.array(values)

np.array(values, dtype='int')

[ 1.53567873e+00  6.65317665e-01  1.11001697e-01 -2.77148667e+00
 -1.23400570e+01 -1.36753213e+01 -8.39585706e+03 -8.59206714e+01
 -2.94466752e+01 -6.79181993e+00  3.47659247e+00  8.81621025e+00
  5.64184071e+00  3.52511907e+00 -2.00211305e-01 -6.44283639e-01
  1.29861858e-01]


array([[[    1],
        [    0],
        [    0],
        [   -2],
        [  -12],
        [  -13],
        [-8395],
        [  -85],
        [  -29],
        [   -6],
        [    3],
        [    8],
        [    5],
        [    3],
        [    0],
        [    0],
        [    0]]])